# Image Stacking Test Playground
This file is a test center for stacking images. The purpose of image stacking is to enhance details by averaging out noise with multiple captures. The goal is to decide whether this will be a useful tool in our machine vision toolkit.

### Import Libraries

In [67]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
import os
from skimage import io, img_as_ubyte

### Define Variables

In [75]:
path01 = '/home/unitx/Desktop/images_capture/outerring_103us'
path02 = '/home/unitx/Desktop/images_capture/innerring_103us'
path03 = '/home/unitx/Desktop/images_capture/inner_outer_103us'

# raw_image01 = io.imread('01/Basler_a2A5328-15ucPRO__40230347__20241111_155016564_0000.tiff')
darkframe = io.imread('stackshield_white_part_DF.png')

### Functions

In [69]:
def stacker(directory):
    images = []
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        # Check if it's an image file by extension (you can adjust as needed)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            img0 = io.imread(file_path)
            img = img0 - darkframe
            images.append(img)
    # Stack images along a new axis to create a 4D array (num_images, height, width, channels)
    stacked_images = np.stack(images, axis=0)
    # print(stacked_images.shape)
    # Calculate the mean across the first axis (i.e., across all images)
    mean_image = np.mean(stacked_images, axis=0).astype(np.uint16)
    # print(mean_image.shape)
    return mean_image

In [106]:
def stacker(directory):
    images01 = []
    images02 = []
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        # Check if it's an image file by extension (you can adjust as needed)
        if 'outer' in filename.lower():
            img0 = io.imread(file_path)
            img = img0 - darkframe
            images01.append(img)
        elif 'inner' in filename.lower():
            img0 = io.imread(file_path)
            img = img0 - darkframe
            images02.append(img)
    # Stack images along a new axis to create a 4D array (num_images, height, width, channels)
    stacked_images01 = np.stack(images01, axis=0)
    stacked_images02 = np.stack(images02, axis=0)
    stacked_images02 = stacked_images02+100
    # Calculate the mean across the first axis (i.e., across all images)
    mean_image01 = np.mean(stacked_images01, axis=0).astype(np.uint16)
    mean_image02 = np.mean(stacked_images02, axis=0).astype(np.uint16)
    return mean_image01, mean_image02

In [107]:
m1, m2 = stacker(path03)

print(np.mean(m1,axis=0))
print(np.mean(m2,axis=0))

print(np.max(m1))
print(np.max(m2))

[[26.03819444 29.03993056 22.20833333]
 [26.26627604 29.04557292 22.20833333]
 [26.26627604 29.04231771 22.33420139]
 ...
 [27.55946181 29.38932292 23.62521701]
 [27.55946181 29.36827257 23.49262153]
 [ 0.77213542  0.99544271  0.44010417]]
[[109.92230903 117.80642361 107.23459201]
 [110.09765625 117.83897569 107.23459201]
 [110.09765625 117.85915799 107.45659722]
 ...
 [111.22200521 117.66731771 108.27669271]
 [111.22200521 117.66189236 108.24153646]
 [ 99.99435764  99.99544271  99.99565972]]
255
254


In [70]:
def stacker2(directory, batch_size=100):
    file_list = [os.path.join(directory, f) for f in os.listdir(directory) 
                 if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    num_files = len(file_list)
    cumulative_image = None
    count = 0

    for i in range(0, num_files, batch_size):
        batch_files = file_list[i:i + batch_size]
        batch_images = []

        for file_path in batch_files:
            img0 = io.imread(file_path)
            img = img0 #- darkframe
            batch_images.append(img)
        
        batch_stack = np.stack(batch_images, axis=0)
        batch_mean = np.median(batch_stack, axis=0)
        
        if cumulative_image is None:
            cumulative_image = batch_mean
        else:
            cumulative_image = (cumulative_image * count + batch_mean * len(batch_files)) / (count + len(batch_files))
        
        count += len(batch_files)
    
    return cumulative_image

In [71]:
def subtractor(filename, darkframe):
    img0 = io.imread(f'{path02}/{filename}')
    img = img0 - darkframe
    return img
    

### Tests

In [72]:
image01 = stacker2(path01)
# print(image01)
# imageX = subtractor('Basler_a2A5328-15ucPRO__40230347__20241111_170449883_0004.tiff',darkframe)

# fig1, axes1 = plt.subplots(1, 2, figsize=(33, 18), gridspec_kw={'width_ratios': [1, 1]})

# im1 = axes1[0].imshow(raw_image01, vmin=0, vmax=4095)
# axes1[0].set_title('raw_image')

# im2 = axes1[1].imshow(image01.astype(int), vmin=0, vmax=4095)
# axes1[1].set_title('stacked_image')

# plt.show()

[[[ 7.4  14.65  4.45]
  [ 6.2  14.65  4.45]
  [ 6.2  14.75  4.5 ]
  ...
  [ 5.85 18.15  7.25]
  [ 5.85 17.3   5.95]
  [ 0.    0.    0.  ]]

 [[ 4.6  16.2   4.45]
  [ 3.75 16.25  4.45]
  [ 3.75 15.1   4.5 ]
  ...
  [ 8.35 18.85  7.25]
  [ 8.35 18.6   5.95]
  [ 0.    0.    0.  ]]

 [[ 4.6  15.5   5.3 ]
  [ 3.75 14.95  5.3 ]
  [ 3.75 14.    4.95]
  ...
  [ 8.35 18.25  7.15]
  [ 8.35 18.    7.05]
  [ 0.    0.    0.  ]]

 ...

 [[ 4.9  14.2   3.  ]
  [ 4.65 14.45  3.  ]
  [ 4.65 12.15  4.  ]
  ...
  [ 4.35 12.75  2.3 ]
  [ 4.35 12.    2.8 ]
  [ 0.    0.    0.  ]]

 [[ 4.9  14.3   0.75]
  [ 4.65 15.    0.75]
  [ 4.65 12.55  3.35]
  ...
  [ 4.35 13.25  3.5 ]
  [ 4.35 12.6   5.05]
  [ 0.    0.    0.  ]]

 [[ 0.    0.    0.  ]
  [ 0.    0.    0.  ]
  [ 0.    0.    0.  ]
  ...
  [ 0.    0.    0.  ]
  [ 0.    0.    0.  ]
  [ 0.    0.    0.  ]]]


### Save Images

In [100]:
def save_as_png(array, filename):
    normalized_array = (array - np.min(array)) / (np.max(array) - np.min(array))
    uint8_array = img_as_ubyte(normalized_array)
    
    io.imsave(filename, uint8_array)

save_as_png(image01, f'stackshield_mean_dandybridge02.png')
# save_as_png(imageX, f'Subtractbook_12bit_{path02}.png')

In [108]:
save_as_png(m1 + m2, f'stackbridge_weird_01.png')